In [ ]:
#These libraries help to interact with the operating system and the runtime environment respectively
import os
import sys

#Model/Training related libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

#Dataloader libraries
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix

import os
import numpy as np
from PIL import Image

# import torch
import torchvision   
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision.transforms import Compose 
from torchvision import transforms

from torchsummary import summary

In [ ]:
#Intall Kaggle API and create kaggle directory
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir .kaggle
#This data is used to login  into your Kaggle account
import json
token = {"username":"samruddhi98","key":"db26269bc2e5ae4d4f8456490e50b5a8"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)


In [ ]:
!chmod 600 /content/.kaggle/kaggle.json

!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config set -n path -v /content

- path is now set to: /content


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Download dataset .npz files from kaggle
!kaggle competitions download -c idl-fall21-hw2p2s1-face-classification-slack

In [ ]:
!unzip competitions/idl-fall21-hw2p2s1-face-classification/idl-fall21-hw2p2s1-face-classification.zip

In [ ]:
class SimpleResidualBlock_resnet(nn.Module):
    def __init__(self, in_channel_size, stride=1):
        super().__init__()
        out_channel_size = int(in_channel_size*stride)
        self.conv1 = nn.Conv2d(in_channel_size, out_channel_size, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv2 = nn.Conv2d(out_channel_size, out_channel_size, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel_size)
        self.bn2 = nn.BatchNorm2d(out_channel_size)
        # self.bn3 = nn.BatchNorm2d(out_channel_size)
        if stride == 1:
            self.shortcut = nn.Identity()
        else:
            self.shortcut = nn.Conv2d(in_channel_size, out_channel_size, kernel_size=1, stride=stride, bias=False)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        shortcut = self.shortcut(x)
        # shortcut = self.bn3(shortcut)
        out = self.relu(out + shortcut)
        return out
    

In [ ]:
# This has hard-coded hidden feature sizes.
# You can extend this to take in a list of hidden sizes as argument if you want.
class ClassificationNetwork(nn.Module):
    def __init__(self, in_features, num_classes, feat_dim = 1000):
        super().__init__()
        
        self.layers = nn.Sequential(
            # nn.Conv2d(in_features, 32, kernel_size=3, stride=1, padding=1, bias=True),
            # nn.BatchNorm2d(32),
            # nn.ReLU(inplace=True),
            nn.Conv2d(in_features, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            SimpleResidualBlock_resnet(64),
            SimpleResidualBlock_resnet(64),
            SimpleResidualBlock_resnet(64),
            SimpleResidualBlock_resnet(64,2),
            SimpleResidualBlock_resnet(128),
            SimpleResidualBlock_resnet(128),
            SimpleResidualBlock_resnet(128),
            SimpleResidualBlock_resnet(128,2),
            SimpleResidualBlock_resnet(256),
            SimpleResidualBlock_resnet(256),
            SimpleResidualBlock_resnet(256),
            SimpleResidualBlock_resnet(256),
            SimpleResidualBlock_resnet(256),
            SimpleResidualBlock_resnet(256,2),
            SimpleResidualBlock_resnet(512),
            SimpleResidualBlock_resnet(512),
            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Flatten(), # the above ends up with batch_size x 64 x 1 x 1, flatten to batch_size x 64
        )

       
        self.linear = nn.Linear(512, feat_dim)
        # nn.init.kaiming_normal(self.linear.weight)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(0.4)
        self.linear_output = nn.Linear(512,num_classes)  
        # nn.init.kaiming_normal(self.linear_output.weight)   

    def forward(self, x, return_embedding=False):
        embedding = self.layers(x) 
        embedding_out = self.relu(self.linear(embedding))
        output = self.linear_output(embedding)
        if return_embedding:
            return embedding_out,output
        else:
            return output   

In [ ]:
# !nvidia-smi

In [ ]:
# Dataloaders

train_transform = Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(p=0.4),
    # transforms.RandomVerticalFlip(p=0.25),
    # transforms.RandomAffine(10),
    transforms.RandomRotation(20)
    # transforms.RandomCrop(32, padding=4),
    
])

test_transform = Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
train_dataset = torchvision.datasets.ImageFolder(root='train_data/', 
                                                 transform=train_transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, 
                                               shuffle=True, num_workers=8)

dev_dataset = torchvision.datasets.ImageFolder(root='val_data/', 
                                               transform=test_transform)
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=128, 
                                             shuffle=False, num_workers=8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# clearing cache to get rid of CUDA out of memory error
torch.cuda.empty_cache()
import gc
try:
    del network
except:
    pass
gc.collect()

# Initializing objects
numEpochs = 100
in_features = 3 # RGB channels
learningRate = 1e-1
weightDecay = 5e-4
num_classes = len(train_dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = ClassificationNetwork(in_features, num_classes, 1000)
network = network.to(device)
# summary(network, (3, 64, 64))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3, verbose=True)

In [ ]:
# load cell (to load any saved model)
###### SKIP THIS CELL IF YOU DO NOT WANT TO LOAD A MODEL #######

epoch = 54
val_acc = 0.9876
PATH = 'model6_Resnet34_epoch%d_val_acc-%0.4f.pt'%(epoch,val_acc)
print('Loading %s model '%(PATH))
checkpoint = torch.load(PATH)
network.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler'])
epochs = checkpoint['epoch']

Loading drive/MyDrive/IDL-HW2P2-S1/model6_Resnet34_epoch54_val_acc-0.9876.pt model 


In [ ]:
# Train!
# summary(network,(3,64,64))
for epoch in range(34,numEpochs):
    
    # Train
    network.train()
    num_correct = 0
    avg_loss = 0.0
    avg_loss_epoch = []
    for batch_num, (x, y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        x, y = x.to(device), y.to(device)
        outputs = network(x)

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()
        avg_loss_epoch.append(loss.item())
        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
        if batch_num % 100 == 99:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, avg_loss/100))
            avg_loss = 0.0
    train_acc = (num_correct / len(train_dataset))
    print('Epoch: {} - Average training loss = {:0.4f}, Training accuracy = {:0.04f}'.format(epoch,sum(avg_loss_epoch)/len(avg_loss_epoch),train_acc))

    # Save
    PATH = 'drive/MyDrive/IDL-HW2P2-S1/model6_Resnet34_epoch%d_val_acc-%0.4f.pt'%(epoch,train_acc)
    torch.save({
        'epoch': numEpochs,
        'model_state_dict': network.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler': scheduler.state_dict(),
        }, PATH)
    
    torch.cuda.empty_cache()
    del x
    del y
    del avg_loss
    gc.collect()

    # Validate
    network.eval()
    num_correct = 0
    avg_val_loss = 0
    for batch_num, (x, y) in enumerate(dev_dataloader):
        x, y = x.to(device), y.to(device)
        outputs = network(x)
        val_loss = criterion(outputs, y.long())
        avg_val_loss += val_loss.item()
        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
    val_acc = (num_correct / len(dev_dataset))
    print('Epoch: {}, Validation Accuracy: {:4f}'.format(epoch,val_acc*100 ))
    scheduler.step(val_acc)
    
    torch.cuda.empty_cache()
    del x
    del y
    del val_loss
    gc.collect()
    

In [ ]:
# Creating a class directory as ImageFolder accepts it
!mkdir test_data/0
!mv test_data/* test_data/0/

# Test dataloader
test_transform = Compose([
    transforms.ToTensor(),
    # transforms.Normalize([0, 0, 0], [1, 1, 1])
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_dataset = torchvision.datasets.ImageFolder(root='test_data/', 
                                               transform=test_transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, 
                                             shuffle=False, num_workers=8)


In [ ]:
# Test
network.eval()
num_correct = 0
predictions = []
for batch_num, (x, y) in enumerate(test_dataloader):
    x, y = x.to(device), y.to(device)
    outputs = network(x)
    outputs = outputs.cpu().detach().numpy()
    outputs = np.argmax(outputs,axis=1)
    predictions.append(outputs)
predictions = np.asarray(predictions).flatten()

In [ ]:
# Inversing the mapping and rearranging the order of predictions

class_mapping = list(train_dataset.class_to_idx.items())
predictions = list(predictions)
preds_list = []
for i in predictions:
    label = [int(j[0]) for j in class_mapping if j[1] == i]
    preds_list.append(label[0])

num = list(np.arange(8000))
num_str = list(map(lambda i:str(i),num))
num_str.sort()
num = list(map(lambda i:int(i),num_str))
pred = {num[i]:preds_list[i] for i in range(len(num))}


final_pred = []
for i in sorted(pred.keys()):
    final_pred.append(pred[i])

In [ ]:
# Saving predictions to csv file
import pandas as pd
filename = 'drive/MyDrive/IDL-HW2P2-S1/submissions_res34.csv'
index = np.arange(0,len(predictions))
index = [str(i)+'.jpg' for i in index]
data = {'id':(index),
        'label':final_pred}
pred_df = pd.DataFrame(data)
pred_df = pred_df.rename_axis('id',axis=1)
print(pred_df)
pred_df.to_csv(filename,index=False)

In [ ]:
# Submitting predictions to kaggle
!kaggle competitions submit -c 'idl-fall21-hw2p2s1-face-classification' -f {filename} -m "Model-res34 submission"


100% 106k/106k [00:09<00:00, 12.0kB/s]
Successfully submitted to IDL-Fall21-HW2P2S1-Face Classification